In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Chronic_obstructive_pulmonary_disease_(COPD)"
cohort = "GSE175616"

# Input paths
in_trait_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)"
in_cohort_dir = "../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE175616"

# Output paths
out_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/GSE175616.csv"
out_gene_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE175616.csv"
out_clinical_data_file = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/clinical_data/GSE175616.csv"
json_path = "../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Clinical study of the effect of combined treatment of aspirin and zileuton on biomarkers of tobacco-related carcinogenesis in current smokers"
!Series_summary	"A chemopreventive effect of aspirin (ASA) on lung cancer risk is supported by epidemiologic and preclinical studies. Zileuton, a 5-LOX inhibitor has single agent activity and adds to the activity of NSAIDs in preclinical models of tobacco carcinogenesis   We hypothesized that COX inhibitor + 5-LOX inhibitor may be more effective than placebo in modulating nasal epithelium gene signatures of tobacco exposure and lung cancer. We conducted a randomized, double-blinded study of low dose ASA plus zileuton vs. double placebo in current smokers to compare modulating effects on nasal epithelium gene expression and arachidonic acid (AA) metabolism. Sixty-three participants were randomized to combined treatment of ASA (81 mg QD) and zileuton (Zyflo CR) two 600 mg extended release tablets BID or place

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the series title and summary, this dataset appears to be focused on gene expression
# from nasal epithelium of smokers. It mentions gene expression signatures, which indicates
# gene expression data should be available.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability for trait (COPD)
# Looking at the sample characteristics dictionary, there's no direct mention of COPD.
# From the background information, it mentions that all participants are current smokers and
# the study mentions "chronic obstructive pulmonary disease" as one of the signatures being studied,
# but it doesn't classify patients as having COPD or not.
trait_row = None  # COPD status is not explicitly available in the data

# 2.1 Data Availability for age
# Age is available at index 6
age_row = 6

# 2.1 Data Availability for gender
# Gender (Sex) is available at index 5
gender_row = 5

# 2.2 Data Type Conversion for trait
def convert_trait(value):
    # Since trait data is not available, this function won't be used
    # But we'll define it for completeness
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # No specific conversion logic since trait data not present
    return None

# 2.2 Data Type Conversion for age
def convert_age(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to integer if possible
    try:
        return int(value)
    except:
        return None

# 2.2 Data Type Conversion for gender
def convert_gender(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    # Convert to binary (0 for female, 1 for male)
    if value == "female":
        return 0
    elif value == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Initial filtering based on trait and gene data availability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(is_final=False, 
                             cohort=cohort, 
                             info_path=json_path, 
                             is_gene_available=is_gene_available,
                             is_trait_available=is_trait_available)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


A new JSON file was created at: ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/cohort_info.json


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Chronic_obstructive_pulmonary_disease_(COPD)/GSE175616/GSE175616_series_matrix.txt.gz


Gene data shape: (20777, 123)
First 20 gene/probe identifiers:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '100033413_at',
       '100033414_at', '100033418_at', '100033420_at', '100033422_at',
       '100033423_at', '100033424_at', '100033425_at', '100033426_at',
       '100033427_at', '100033428_at', '100033430_at', '100033431_at',
       '100033432_at', '100033433_at', '100033434_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers in the gene expression data
# These identifiers (e.g., '100009676_at', '10000_at') are Affymetrix probe IDs
# They're not standard human gene symbols and need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Let's look for platform information in the SOFT file to understand the annotation better
print("\nSearching for platform information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    for i, line in enumerate(f):
        if '!Series_platform_id' in line:
            print(line.strip())
            break
        if i > 100:  # Limit search to first 100 lines
            print("Platform ID not found in first 100 lines")
            break

# Check if the SOFT file includes any reference to gene symbols
print("\nSearching for gene symbol information in SOFT file:")
with gzip.open(soft_file, 'rt') as f:
    gene_symbol_lines = []
    for i, line in enumerate(f):
        if 'GENE_SYMBOL' in line or 'gene_symbol' in line.lower() or 'symbol' in line.lower():
            gene_symbol_lines.append(line.strip())
        if i > 1000 and len(gene_symbol_lines) > 0:  # Limit search but ensure we found something
            break
    
    if gene_symbol_lines:
        print("Found references to gene symbols:")
        for line in gene_symbol_lines[:5]:  # Show just first 5 matches
            print(line)
    else:
        print("No explicit gene symbol references found in first 1000 lines")

# Look for alternative annotation files or references in the directory
print("\nChecking for additional annotation files in the directory:")
all_files = os.listdir(in_cohort_dir)
print([f for f in all_files if 'annotation' in f.lower() or 'platform' in f.lower() or 'gpl' in f.lower()])



Gene annotation preview:
Columns in gene annotation: ['ID', 'SPOT_ID', 'DESCRIPTION']
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'SPOT_ID': ['1', '10', '100', '1000', '10000'], 'DESCRIPTION': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2', 'adenosine deaminase', 'cadherin 2', 'AKT serine/threonine kinase 3']}

Searching for platform information in SOFT file:
Platform ID not found in first 100 lines

Searching for gene symbol information in SOFT file:


No explicit gene symbol references found in first 1000 lines

Checking for additional annotation files in the directory:
[]


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene annotation contain identifiers and gene symbols
# From the preview, we can see:
# - 'ID' column in gene_annotation contains identifiers like '1_at', '10_at', which match the format in gene_data
# - 'DESCRIPTION' column appears to contain the gene symbols/names like 'alpha-1-B glycoprotein'

# 2. Extract gene mapping dataframe with the relevant columns
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='DESCRIPTION')
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("Gene mapping preview:")
print(preview_df(mapping_df, n=5))

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
# The apply_gene_mapping function will:
# - Divide probe values equally among mapped genes
# - Sum values across all probes for each gene
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(gene_data.index[:10].tolist())

# Normalize gene symbols to standard format
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10].tolist())

# Save the processed gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (20770, 2)
Gene mapping preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'Gene': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2', 'adenosine deaminase', 'cadherin 2', 'AKT serine/threonine kinase 3']}
Gene expression data shape after mapping: (2377, 123)
First few gene symbols after mapping:
['A-', 'A-52', 'A0', 'A1', 'A1-', 'A10', 'A11', 'A12', 'A13', 'A14']


Gene expression data shape after normalization: (1639, 123)
First few normalized gene symbols:
['A4GALT', 'AAA1', 'AAR2', 'ABCC11', 'ABCE1', 'ABO', 'ABR', 'ABRA', 'ACD', 'ACOXL']


Gene expression data saved to ../../output/preprocess/Chronic_obstructive_pulmonary_disease_(COPD)/gene_data/GSE175616.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data 
# (This was already done in the previous step, so no need to repeat)
print(f"Normalized gene data shape: {gene_data.shape}")

# 2. Examine the sample IDs in the gene expression data to understand the structure
print("Gene data column names (sample IDs):")
print(gene_data.columns[:5])  # Print first 5 for brevity

# Since trait information is not available (trait_row is None), we skip clinical feature extraction
# and proceed directly with gene expression data
print("\nNo trait information available for COPD in this dataset")

# Create clinical features with age and gender information only
sample_ids = gene_data.columns.tolist()

# Extract age data if available
clinical_features_list = []
if age_row is not None:
    age_data = get_feature_data(clinical_data, age_row, 'Age', convert_age)
    clinical_features_list.append(age_data)
    
# Extract gender data if available
if gender_row is not None:
    gender_data = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender)
    clinical_features_list.append(gender_data)

# If we have any clinical features, create the dataframe and link with gene data
if clinical_features_list:
    clinical_features = pd.concat(clinical_features_list, axis=0)
    print(f"Created clinical features dataframe with demographic info only:")
    print(f"Shape: {clinical_features.shape}")
    print(clinical_features.iloc[:, :5])  # Show first 5 columns
    
    # Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
    print(f"\nLinked data shape before handling missing values: {linked_data.shape}")
    
    # Handle missing values - no trait column, so we use the first column 
    # for the mandatory trait parameter in handle_missing_values
    first_col = linked_data.columns[0]
    linked_data_clean = handle_missing_values(linked_data, first_col)
    print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")
    
    # Since we don't have trait information, we don't evaluate trait bias
    # We'll just check for bias in demographic features
    # Creating a temporary dataframe for bias checking with the first column as 'placeholder_trait'
    temp_df = linked_data_clean.copy()
    temp_df['placeholder_trait'] = 1  # All samples get the same value
    is_biased, linked_data_clean = judge_and_remove_biased_features(temp_df, 'placeholder_trait')
    linked_data_clean = linked_data_clean.drop('placeholder_trait', axis=1, errors='ignore')
else:
    # If no clinical features, just use gene expression data
    print("No clinical features available, proceeding with gene expression data only")
    linked_data = gene_data.T  # Transpose to match expected format
    linked_data_clean = linked_data
    is_biased = True  # Mark as biased since no trait information

# Conduct final quality validation
note = "Dataset contains gene expression data from nasal epithelium of smokers in a study of combined aspirin and zileuton treatment. No COPD trait information available."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True, 
    is_trait_available=False,  # No trait information for COPD
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable due to quality issues - linked data not saved")

Normalized gene data shape: (1639, 123)
Gene data column names (sample IDs):
Index(['GSM5342551', 'GSM5342552', 'GSM5342553', 'GSM5342554', 'GSM5342555'], dtype='object')

No trait information available for COPD in this dataset
Created clinical features dataframe with demographic info only:
Shape: (2, 123)
        GSM5342551  GSM5342552  GSM5342553  GSM5342554  GSM5342555
Age           35.0        35.0        35.0        63.0        63.0
Gender         1.0         1.0         1.0         1.0         1.0

Linked data shape before handling missing values: (123, 1641)


Linked data shape after handling missing values: (123, 1641)


Quartiles for 'placeholder_trait':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'placeholder_trait' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 47.5
  50% (Median): 52.0
  75%: 57.0
Min: 35.0
Max: 72.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 60 occurrences. This represents 48.78% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset deemed not usable due to quality issues - linked data not saved
